In [1]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from PIL import Image
import numpy as np
modes = [AES.MODE_ECB, AES.MODE_CBC, AES.MODE_OFB, AES.MODE_CFB, AES.MODE_CTR]
is_iv_needed_modes = [False, True, True, True, False]
files = ['maly_pingwinek.png', 'sredni_pingwinek.png', 'duzy_pingwinek.png']
key = get_random_bytes(16)

In [13]:
def byte_xor(bytes1: bytes, bytes2: bytes):
    xored = b''
    for b1, b2 in zip(bytes1, bytes2):
        newi = b1 ^ b2
        newb = newi.to_bytes()
        xored += newb
    return xored

In [2]:
for mode, is_iv_needed in zip(modes, is_iv_needed_modes):
    cipher = None
    if is_iv_needed:
        iv = get_random_bytes(AES.block_size)
        cipher = AES.new(key=key, mode=mode, iv=iv)
    else:
        cipher = AES.new(key=key, mode=mode)
    for file in files:
        img = Image.open(file)
        bytes = img.tobytes()
        bytes = pad(bytes, AES.block_size)
        en_bytes = cipher.encrypt(bytes)
        with open('pingwiny/{}_{}'.format(mode, file[:-4]), mode='wb') as f:
            f.write(en_bytes)

In [16]:
for mode, is_iv_needed in zip(modes, is_iv_needed_modes):
    cipher = None
    if is_iv_needed:
        iv = get_random_bytes(AES.block_size)
        cipher = AES.new(key=key, mode=mode, iv=iv)
        err_cipher = AES.new(key=key, mode=mode, iv=iv)
    else:
        cipher = AES.new(key=key, mode=mode)
        err_cipher = AES.new(key=key, mode=mode)
    for file in files:
        img = Image.open(file)
        byte_img = img.tobytes()
        byte_img = pad(byte_img, AES.block_size)
        en_bytes = cipher.encrypt(byte_img)
        error = bytes.fromhex('0001')
        err_bytes = byte_xor(en_bytes[:2], error) + en_bytes[2:]
        de_bytes = err_cipher.decrypt(err_bytes)
        with open('pingwiny/err_file_{}_{}'.format(mode, file[:-4]), mode='wb') as f:
            f.write(de_bytes)

In [17]:
err_key = byte_xor(key, (bytes.fromhex('0001') + b'\0' * 14))
for mode, is_iv_needed in zip(modes, is_iv_needed_modes):
    cipher = None
    if is_iv_needed:
        iv = get_random_bytes(AES.block_size)
        cipher = AES.new(key=key, mode=mode, iv=iv)
        err_cipher = AES.new(key=err_key, mode=mode, iv=iv)
    else:
        cipher = AES.new(key=key, mode=mode)
        err_cipher = AES.new(key=err_key, mode=mode)
    for file in files:
        img = Image.open(file)
        bytes = img.tobytes()
        bytes = pad(bytes, AES.block_size)
        en_bytes = cipher.encrypt(bytes)
        de_bytes = err_cipher.decrypt(en_bytes)
        with open('pingwiny/err_key_{}_{}'.format(mode, file[:-4]), mode='wb') as f:
            f.write(de_bytes)